<a href="https://colab.research.google.com/github/ransjnr/building-ai-agents-with-autogen/blob/main/Research_Assistant_for_Marketing_Videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Research Assistant for Marketing Videos

###creating agents for the assistant


In [1]:
!pip install ag2
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.0 MB/s eta 0:00:00


In [2]:
##import requried libraries
import os
from autogen import ConversableAgent, UserProxyAgent
from openai import AzureOpenAI
from google.colab import userdata

###setting up credentials

In [3]:
endpoint = userdata.get("AZURE_OPENAI_ENDPOINT")
model_name = "o4-mini"
deployment = "o4-mini-deployment"

subscription_key = userdata.get("AZURE_OPENAI_KEY")
api_version = "2024-12-01-preview"

###llm configuration

In [4]:
llm_config = {
    "config_list": [
        {
            "model": deployment,
            "base_url": endpoint,
            "api_type": "azure",
            "api_key": subscription_key,
            "api_version": api_version,
        }
    ]
}

###Manager Proxy Agent

In [5]:
manager_proxy_agent = UserProxyAgent(
    name="Manager",
    system_message="You are a marketing campaign manager. Your role is to provide campaign topics and evaluate the ideas and scripts generated by the creative team.",
    human_input_mode="NEVER",
    code_execution_config=False,
    llm_config=llm_config,
)

###Idea Generator Agent

In [6]:
idea_generator = ConversableAgent (
    name="IdeaGenerator",
    system_message='''You are a creative marketing expert. Ypur role is to:
    1. Generate innovative campaign ideas based on the given topic
    2. Suggest potential storylines and characters for video content
    3. Provide a brief outline for each idea
    Present 3-5 unique ideas for each request.''',
    llm_config=llm_config,
    human_input_mode="NEVER",
)

###Script Writer Agent

In [7]:
script_writer = ConversableAgent(
    name="ScriptWriter",
    system_message='''You are an expert script writer for marketing videos. Your role is to:
    1. Develop full scripts based on the provided campaign ideas
    2. Include dialogue, scene descriptions, and camera directions
    3. Ensure the script aligns with the campaign objectives and target audience
    Aim for scripts of 2-3 minutes each.''',
    llm_config=llm_config,
    human_input_mode="NEVER",

)

###Script Reviewer Agent

In [8]:
script_reviewer = ConversableAgent(
    name="ScriptReviewer",
    system_message='''You are a critical script reviewer. Your role is to:
    1. Analyze scripts for effectiveness, engagement, and alignment with campaign goals
    2. Provide constructive feedback on dialogue, pacing, and overall structure
    3. Suggest improvements or alternatives where neccessary
    4. Ensure the script adheres to brand guidelines and target audience preferences
    Be thorough and specific in your feedback.''',
    llm_config=llm_config,
    human_input_mode="NEVER",
)

### Creating tasks for the agent

In [9]:
chats = [
    {
        "sender": manager_proxy_agent,
        "recipient": idea_generator,
        "message":
            "Hello, I need creative ideas for our new marketing campaign. "
            "Our product is eco-friendly home cleaning products. "
            "Could you generate 3-5 innovative campaign ideas, including potential "
            "storylines and characters, and a brief outline for video content? ",
        "summary_method": "reflection_with_llm",
        "summary_args":{
            "summary_prompt": "Summarize the generated ideas "
                            "into a JSON object: "
                            "{'idea1': {'concept': '', 'storyline': '', 'characters': ''}, "
                            "'idea2': {...}, 'idea3': {...}}",

        },
        "max_turns": 2,
        "clear_history":True,
    },
    {
        "sender": manager_proxy_agent,
        "recipient": script_writer,
        "message":
            "Great ideas! Let's develop a script for the following concept: "
            "[INSERT SELECTED IDEA HERE]. "
            "Please write a full script including dialogue, scene descriptions, "
            "and camera directions. Aim for a 2-3 minute video. ",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Provide a brief summary of the script "
                            "as a JSON object: "
                            "{'title': '', 'main_characters': [], 'key_scenes': [], "
                            "'estimated_duration': ''}",
        },
        "max_turns": 2,
        "clear_history":True,

    },
    {
        "sender": manager_proxy_agent,
        "recipient": script_reviewer,
        "message": "Please review the following script: [INSERT SCRIPT HERE]. "
                   "Analyze it for effectiveness, engagement, and alignment with "
                   "campaign goals. Provide constructive feedback "
                   "feedback and suggest improvements. ",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Sumarize the review points "
                              "as a JSON object: "
                              "{'strengths': [], 'weaknesses': [], 'suggestions': []}",
        },
    },
    {
        "sender": manager_proxy_agent,
        "recipient": manager_proxy_agent,
        "message": "Based on the script and review, decide if the script is ready "
                   "for production or need revisions. If revisions are needed, "
                   "specify the key areas to address.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Summarize the decision "
                            "as a JSON object: "
                            "{'decision': 'ready'/'needs_revision',"
                            "'key_revision_areas': []}"
        },

    }
]

###Start researching

In [10]:
from autogen import initiate_chats
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Manager (to IdeaGenerator):

Hello, I need creative ideas for our new marketing campaign. Our product is eco-friendly home cleaning products. Could you generate 3-5 innovative campaign ideas, including potential storylines and characters, and a brief outline for video content? 

--------------------------------------------------------------------------------
[autogen.oai.client: 05-13 17:05:34] {696} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


IdeaGenerator (to Manager):

Here are five distinct campaign concepts for your eco-friendly home cleaning line. Each includes a working title, key characters, core storyline and a 30–60-second video outline.

1. “The Eco-Squad to the Rescue”  
   • Premise: A family of green-powered superheroes tackles dirt and chemical waste, restoring homes – and planet health – in every episode.  
   • Characters:  
     – Captain Clean (mom figure with “spray-shield” powers)  
     – Lady Lather (dad-figure who conjures suds from thin air)  
     – Sidekicks Tommy & Emma (kids who scout trouble spots with magnifying-glass drones)  
   • Video Outline (60s):  
     1. Opening: Alert siren, house overrun by toxic “dirt monsters.”  
     2. Call-out: Eco-Squad assembles (costumes glow green).  
     3. Action: Each family member uses your product to defeat a different “monster” (grime, grease, mold).  
     4. Finale: Home sparkles, backyard blooms, family high-five under a “100% Biodegradable” badge.

Manager (to IdeaGenerator):

Here’s my evaluation of each concept—strengths, potential gaps, and tweaks to maximize impact—followed by a recommendation on next steps.

1. “The Eco-Squad to the Rescue”  
Strengths  
 • Family-friendly, hero imagery instantly grabs attention.  
 • Visually rich (costumes, “dirt monsters”) lends itself well to TV spots and TikTok skits.  
 • Clear brand positioning: cleaning = saving the planet.  
Opportunities  
 • Tone: avoid feeling too “campy” or comic-book cliché—keep the humor light and relatable.  
 • Product focus: weave in a quick “how it works” moment (e.g. a microscopic before/after).  
 • Extension: invite real families to share their “Eco-Squad wins” via UGC.  

2. “Green Makeover Challenge”  
Strengths  
 • Highly social-native—perfect for YouTube, Reels and challenge culture.  
 • Competitive format delivers built-in suspense and shareability.  
 • Demonstrates performance (time-lapse) and sustainability metrics (air-quality, residue).  
Op

IdeaGenerator (to Manager):

Great—let’s green-light the two priority concepts and set “Nature’s Little Helpers” in development as a supportive asset suite.

1. Green Makeover Challenge (Social-First Series)  
  • Objective: Drive UGC participation and product trials by positioning your line as the go-to eco swap.  
  • Key Deliverables:  
    – 6× 45-sec challenge videos (Kitchen, Bath, Living Room rounds)  
    – 30× 15-sec cutdowns for Reels/Stories/TikTok  
    – Branded “Green Meter” overlays + on-screen CTAs  
    – Influencer micro-kits (10 eco-creators each)  
  • Timeline:  
    Weeks 1–2: Script & obstacle design workshop  
    Weeks 3–4: Talent scouting + kitchen set build  
    Week 5: Shoot & in-house edit  
    Week 6: Social launch & seeding  
  • Budget Range (est.): $60–80K per pilot market—including talent fees, production, post and UGC amplification.  
  • Channels: Instagram Reels, TikTok, YouTube Shorts, Facebook Stories

2. The Microbe Detective (Digital Pre-Roll 


********************************************************************************
Starting a new chat....

********************************************************************************
Manager (to ScriptWriter):

Great ideas! Let's develop a script for the following concept: [INSERT SELECTED IDEA HERE]. Please write a full script including dialogue, scene descriptions, and camera directions. Aim for a 2-3 minute video. 
Context: 
{
  "idea1": {
    "concept": "Green Makeover Challenge",
    "storyline": "Two families go head-to-head converting from conventional to eco-friendly cleaning in three timed rounds (kitchen, bathroom, living room), demonstrating product performance and sustainability metrics.",
    "characters": "Host (energetic influencer), Family A, Family B"
  },
  "idea2": {
    "concept": "The Microbe Detective",
    "storyline": "A film-noir style detective and his dog sidekick hunt down and expose hidden germ-gangsters in household hotspots, using eco-friendly spray 

...

ScriptWriter (to Manager):

Title: “The Microbe Detective”  
Length: ~2:30  
Tone: Playful film-noir with eco-credentials  

––––––––––––––––––––––––––––––––––––––––  
SCENE 1: “THE KITCHEN CRIME SCENE”  
INT. KITCHEN – NIGHT  
Atmosphere: Dim lamplight, heavy shadows, noir jazz under-score.  

Sound design: Slow footsteps on tile. Soft hum of refrigerator.  

CAMERA:  
• WIDE SHOT of cluttered countertop.  
• CUT TO close-up of Detective Green’s polished derby and trench coat.  

DETECTIVE GREEN (V.O., gravelly):  
In this town, every countertop’s got a secret. And I’m the guy who cracks the case.  

• WHIP-PAN to DETECTIVE GREEN, mid-30s, tweed jacket, holding a UV lamp.  
• CLOSE-UP: UV LAMP BUZZ activates, revealing neon “germ-gang” graffiti across the cutting board.  

DETECTIVE GREEN (whispers):  
Case #001: The Midnight Meat Muck.  

Enter WATSON (a sleek terrier in a tiny trench coat).  

WATSON (dry, British accent):  
Sir, those microbes are as slippery as a greased pickpocke


********************************************************************************
Starting a new chat....

********************************************************************************
Manager (to ScriptReviewer):

Please review the following script: [INSERT SCRIPT HERE]. Analyze it for effectiveness, engagement, and alignment with campaign goals. Provide constructive feedback feedback and suggest improvements. 
Context: 
{
  "idea1": {
    "concept": "Green Makeover Challenge",
    "storyline": "Two families go head-to-head converting from conventional to eco-friendly cleaning in three timed rounds (kitchen, bathroom, living room), demonstrating product performance and sustainability metrics.",
    "characters": "Host (energetic influencer), Family A, Family B"
  },
  "idea2": {
    "concept": "The Microbe Detective",
    "storyline": "A film-noir style detective and his dog sidekick hunt down and expose hidden germ-gangsters in household hotspots, using eco-friendly spray and UV r

ScriptReviewer (to Manager):

Here’s a detailed critique of your “The Microbe Detective” script‐outline, focused on effectiveness, engagement, pacing and alignment with your eco-cleaning campaign. Wherever possible, I’ve noted actionable tweaks.

1. OVERALL CONCEPT & TONE  
• Strengths:  
  – Film-noir framing gives you an instantly recognizable style that stands out from typical cleaning ads.  
  – Detective + talking dog is playful and lends itself to humorous banter.  
  – The “cases” metaphor works well to showcase hidden germs and then “solve” each with your product.  
• Risks:  
  – Noir tropes (dark lighting, sarcastic asides) can undercut the eco-friendly, family-friendly positioning if over-stressed.  
  – If the dog is too chatty or the detective too cliché, viewers may remember the jokes but miss the sustainability message.  

Recommendation: Lean into a “light-noir” palette—moody music and silhouettes, but upbeat voice–overs and pops of green/blue on screen to signal “eco.”


********************************************************************************
Starting a new chat....

********************************************************************************
Manager (to Manager):

Based on the script and review, decide if the script is ready for production or need revisions. If revisions are needed, specify the key areas to address.
Context: 
{
  "idea1": {
    "concept": "Green Makeover Challenge",
    "storyline": "Two families go head-to-head converting from conventional to eco-friendly cleaning in three timed rounds (kitchen, bathroom, living room), demonstrating product performance and sustainability metrics.",
    "characters": "Host (energetic influencer), Family A, Family B"
  },
  "idea2": {
    "concept": "The Microbe Detective",
    "storyline": "A film-noir style detective and his dog sidekick hunt down and expose hidden germ-gangsters in household hotspots, using eco-friendly spray and UV reveals to solve each case.",
    "characters": "Detec

Manager (to Manager):

The “Microbe Detective” spot is not yet production-ready—it needs a focused pass on tone, pacing and clarity. Key areas to address before you roll cameras:

1. Tone Consistency  
   • Dial back heavy film-noir tropes so the spot stays family-friendly and upbeat.  
   • Use a “light-noir” look (moody grays + brand green) with an energetic, warm VO rather than brooding monologues.  

2. Dialogue & Character Banter  
   • Trim Watson to 2–3 zingers total; Detective Green gets one solid quip per scene.  
   • Eliminate any throwaway lines that don’t advance the sustainability message.  

3. Scene Timing & Pacing  
   • Lock in your run-of-show: 10s hook, 35s kitchen, 30s bathroom, 35s living room, 25s eco-montage, 15s close.  
   • Watch for drag in the montage—aim for under 25 seconds of simple iconography + one-line captions.  

4. Scene Focus  
   • Kitchen: split-screen before/after + punchy “germ vanish” SFX.  
   • Bathroom: render the “bogeyman” as friendly gr

In [12]:
for chat_result in chat_results:
  print(chat_result.summary)
  print("\n")

{
  "idea1": {
    "concept": "Green Makeover Challenge",
    "storyline": "Two families go head-to-head converting from conventional to eco-friendly cleaning in three timed rounds (kitchen, bathroom, living room), demonstrating product performance and sustainability metrics.",
    "characters": "Host (energetic influencer), Family A, Family B"
  },
  "idea2": {
    "concept": "The Microbe Detective",
    "storyline": "A film-noir style detective and his dog sidekick hunt down and expose hidden germ-gangsters in household hotspots, using eco-friendly spray and UV reveals to solve each case.",
    "characters": "Detective Green (tweed-jacket sleuth), Watson (talking pet dog)"
  },
  "idea3": {
    "concept": "Nature’s Little Helpers",
    "storyline": "Personified natural ingredients team up to defeat animated dirt-bugs in quick, colorful battles, showcasing each ingredient’s eco-benefit.",
    "characters": "Lemon Lulu (citrus sprayer), Baking-Bob (odor-neutralizing powder), Bamboo Bre

In [13]:
for chat_result in chat_results:
  print(chat_result.cost)
  print("\n")

{'usage_including_cached_inference': {'total_cost': 0, 'o4-mini': {'cost': 0, 'prompt_tokens': 5864, 'completion_tokens': 4035, 'total_tokens': 9899}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'o4-mini': {'cost': 0, 'prompt_tokens': 5864, 'completion_tokens': 4035, 'total_tokens': 9899}}}


{'usage_including_cached_inference': {'total_cost': 0, 'o4-mini': {'cost': 0, 'prompt_tokens': 4228, 'completion_tokens': 3817, 'total_tokens': 8045}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'o4-mini': {'cost': 0, 'prompt_tokens': 4228, 'completion_tokens': 3817, 'total_tokens': 8045}}}


{'usage_including_cached_inference': {'total_cost': 0, 'o4-mini': {'cost': 0, 'prompt_tokens': 3661, 'completion_tokens': 4188, 'total_tokens': 7849}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'o4-mini': {'cost': 0, 'prompt_tokens': 3661, 'completion_tokens': 4188, 'total_tokens': 7849}}}


{'usage_including_cached_inference': {'total_cost': 0, 'o4-mini': {'cost': 0, 'prompt

###Add web search ability to research assistant

In [14]:
!pip install "tavily-python"

In [18]:
from typing import Annotated
from tavily import TavilyClient
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, register_function

###tavily as a tool for searching the web

In [20]:
tavily = TavilyClient(api_key=userdata.get("TAVILY_API_KEY"))

def search_tool(query: Annotated[str, "The search query"]) -> Annotated[str, "The search results"]:
  return tavily.get_search_context(query=query, search_depth="advanced")

###Adding a ReAct prompt

In [21]:
ReAct_prompt = """
Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
"""

# Define the ReAct prompt message. Assuming a "question" field is present in the context
def react_prompt_message(sender, recipient, context):
  return ReAct_prompt.format(input=context["question"])

###Define agents

In [22]:
user_proxy = UserProxyAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config=False
)

research_assistant = AssistantAgent(
    name="Assistant",
    system_message="""
                  You are a helpful research assistant who has the ability to seach the web using the provided tools.
                  Only use the tools you have been provided with. Reply TERMINATE when the task is done.
                  """,
    llm_config=llm_config,
)

###Register the search tool

In [23]:
register_function(
    search_tool,
    caller= research_assistant,
    executor=user_proxy,
    name="search_tool",
    description="Search the web for the given query",
)

###Initiate the chat

In [24]:
user_proxy.initiate_chat(
    research_assistant,
    message=react_prompt_message,
    question="Who wown the T20 cricket world cup in 2024",
)


User (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: Who wown the T20 cricket world cup in 2024


--------------------------------------------------------------------------------
[autogen.oai.client: 05-13 18:06:35] {696} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Assistant (to User):

***** Suggested tool call (call_Ak22rrJ11VuR59zIhIFra97H): search_tool *****
Arguments: 
{"query": "2024 ICC Men's T20 World Cup winner"}
****************************************************************************

--------------------------------------------------------------------------------
Replying as User. Provide feedback to Assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION search_tool...
Call ID: call_Ak22rrJ11VuR59zIhIFra97H
Input arguments: {'query': "2024 ICC Men's T20 World Cup winner"}
User (to Assistant):

***** Response from calling tool (call_Ak22rrJ11VuR59zIhIFra97H) *****
[{"url": "https://en.wikipedia.org/wiki/2024_Men%27s_T20_World_Cup_final", "content": "The 2024 ICC Men's T20 World Cup final was a Twenty20 International (T20I) cricket match played at Kensington Oval in Bridgetown, Barbados on 29 June 2024 to

Assistant (to User):

Question: Who won the T20 cricket World Cup in 2024  
Thought: I should look up the winner of the 2024 ICC Men’s T20 World Cup.  
Action: search_tool  
Action Input: {"query": "2024 ICC Men's T20 World Cup winner"}  
Observation: The final was played between India and South Africa on 29 June 2024 at Kensington Oval, Bridgetown. India batted first and scored 176/7; South Africa managed 169/8. India won by 7 runs.  
Thought: I now know the final answer.  
Final Answer: India.

--------------------------------------------------------------------------------
Replying as User. Provide feedback to Assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit

>>>>>>>> TERMINATING RUN (7bd0f320-9fd6-4125-a6fb-357ab5bec858): User requested to end the conversation


ChatResult(chat_id=None, chat_history=[{'content': '\nAnswer the following questions as best you can. You have access to tools provided.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take\nAction Input: the input to the action\nObservation: the result of the action\n... (this process can repeat multiple times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\nQuestion: Who wown the T20 cricket world cup in 2024\n', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_Ak22rrJ11VuR59zIhIFra97H', 'function': {'arguments': '{"query": "2024 ICC Men\'s T20 World Cup winner"}', 'name': 'search_tool'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '[{"url": "https://en.wikipedia.org/wiki/2024_Men%27s_T20_World_Cup_final", "content": "The 2024 ICC Men\'s T20 World Cup final was a Twenty20 I